In [ ]:
pip install transformers

     |████████████████████████████████| 2.5MB 24.8MB/s 
     |████████████████████████████████| 3.3MB 39.1MB/s 
     |████████████████████████████████| 901kB 36.9MB/s 


In [ ]:
import torch
import transformers
import pandas as pd
import numpy as np

In [ ]:
from tqdm import notebook
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_excel("Рефлексия.xls") #загружаем базу отзывов

In [ ]:
df_test = pd.read_excel("Тест_отзывы.xlsx")

In [ ]:
df1 = df

In [ ]:
df1

,COURSE_ID,Merged_data,Result
0,34,Неделя 3 затрагивала вопросы оптимального упра...,0.0
1,288,Буду применять в работе. Делиться опытом с ко...,1.0
2,288,"разрабатывать цифровые материалы, подготавлива...",NaN
3,288,Максимум. В повседневной работе с заявителями ...,1.0
4,288,"Могу поделиться с коллегами знаниями, помочь р...",NaN
...,...,...,...
39451,7776,Данные знания и навыки мне пригодятся как в ра...,NaN
39452,7776,Планирую создать сайт для запуска своего старт...,NaN
39453,7776,Базовые знания помогли мне разобраться в основ...,NaN
39454,7776,"На данный курс я записалась, чтобы получить на...",NaN


In [ ]:
df_test = shuffle(df_test)

In [ ]:
df_test.head()

,COURSE_ID,Merged_data,Result
30,6333,Полученные знания и навыки будут применяться п...,1
41,6449,"К сожалению только общие понятия раскрыты, так...",0
49,6520,Я буду проводить уроки для своих учеников (вос...,1
42,6456,Я планирую использовать полученные знания в ра...,1
10,442,Так как я в настоящее время нахожусь в поиске ...,1


In [ ]:
df.head()

,COURSE_ID,Merged_data,Result
0,34,Неделя 3 затрагивала вопросы оптимального упра...,0.0
1,288,Буду применять в работе. Делиться опытом с ко...,1.0
2,288,"разрабатывать цифровые материалы, подготавлива...",NaN
3,288,Максимум. В повседневной работе с заявителями ...,1.0
4,288,"Могу поделиться с коллегами знаниями, помочь р...",NaN


In [ ]:
df_test.shape

(60, 3)

In [ ]:
tokenizer = transformers.BertTokenizer(
    vocab_file='vocab.txt') #создаем токенайзер

tokenized = df_test['Merged_data'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
padded

array([[   101,  65268,    142, ...,      0,      0,      0],
       [   101,  35676,  31367, ...,      0,      0,      0],
       [   101,  53419,  59004, ...,      0,      0,      0],
       ...,
       [   101,  27799,  26661, ...,      0,      0,      0],
       [   101,  27799,  80226, ...,      0,      0,      0],
       [   101,   8074, 110607, ...,      0,      0,      0]])

In [ ]:
padded_2 = []
for i in range(len(padded)):
    padded_2.append(padded[i][:100])

In [ ]:
padded_2 = np.array(padded_2)

In [ ]:
padded_2

array([[   101,  65268,    142, ...,    108,    132,    102],
       [   101,  35676,  31367, ...,      0,      0,      0],
       [   101,  53419,  59004, ...,      0,      0,      0],
       ...,
       [   101,  27799,  26661, ...,  20359,   7591,   7119],
       [   101,  27799,  80226, ...,   2067,  64378,  33351],
       [   101,   8074, 110607, ...,  76146,  16753,   1758]])

Токенизируем основной датасет


In [ ]:
tokenized = df1['Merged_data'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

In [ ]:
tokenized.dtype

dtype('O')

In [ ]:
tokenized.loc[:100]

0      [101, 65268, 142, 34009, 19060, 16365, 85487, ...
1      [101, 35676, 31367, 845, 11321, 132, 67282, 38...
2      [101, 53419, 59004, 17414, 128, 69539, 2110, 1...
3      [101, 33705, 132, 845, 30004, 1438, 852, 11321...
4      [101, 31424, 78711, 869, 36873, 67297, 128, 19...
                             ...                        
96     [101, 28401, 41103, 48790, 20150, 10781, 851, ...
97     [101, 877, 31943, 128, 1997, 27799, 26661, 851...
98     [101, 27799, 40296, 52615, 16740, 11471, 852, ...
99     [101, 2306, 128, 2739, 877, 13831, 898, 845, 2...
100    [101, 27799, 26661, 851, 40296, 13123, 52615, ...
Name: Merged_data, Length: 101, dtype: object

In [ ]:
tokenized_truncated = []
for i in range(len(tokenized)):
    tokenized_truncated.append(tokenized[i][:500])

In [ ]:
tokenized_truncated = np.array(tokenized_truncated)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
tokenized_truncated.shape

(39456,)

In [ ]:
max_len = 0
for i in tokenized_truncated:
    if len(i) > max_len:
        max_len = len(i)
        

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized_truncated])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
padded.shape

(39456, 500)

In [ ]:
max_len

500

In [ ]:
padded.shape

(1000,)

In [ ]:
39456/2

19728.0

In [ ]:
padded.shape

(60, 498)

In [ ]:
config = transformers.BertConfig.from_json_file(
    'bert_config.json')
model = transformers.BertModel.from_pretrained(
    'pytorch_model.bin', config=config)

Some weights of the model checkpoint at pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuBert качать по ссылке: http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html версия pytorch (не удалось приложить в гит файл pytorch_model.bin из-за очень большого размера) 

In [ ]:
batch_size = 1
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

Векторизуем основной датасет


In [ ]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [ ]:
embeddings

In [ ]:
X=np.concatenate(embeddings)

In [ ]:
features = np.concatenate(embeddings)
target = df_test['Result']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.5, random_state=64)  

model_lr = LogisticRegression(solver='lbfgs', class_weight='balanced', n_jobs=2, max_iter=1000)
model_lr.fit(features_train, target_train)
predictions = model_lr.predict(features_test)
model_lr.score(features_test, target_test)

0.8

In [ ]:
from sklearn.metrics import f1_score
f1_score(predictions, target_test, average='macro')

0.7619047619047619

In [ ]:
pred = model_lr.predict(X)

In [ ]:
df1['predicted_values'] = pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1

,COURSE_ID,Merged_data,Result,predicted_values
0,34,Неделя 3 затрагивала вопросы оптимального упра...,0.0,1
1,288,Буду применять в работе. Делиться опытом с ко...,1.0,1
2,288,"разрабатывать цифровые материалы, подготавлива...",NaN,0
3,288,Максимум. В повседневной работе с заявителями ...,1.0,1
4,288,"Могу поделиться с коллегами знаниями, помочь р...",NaN,1
...,...,...,...,...
995,482,"По возможности, использовать полученные знания...",NaN,1
996,482,Планирую использовать с целью модернизации и у...,NaN,1
997,482,Полученные знания планирую использовать на пра...,NaN,1
998,482,Полученные мною знания помогут мне повысить бе...,NaN,0


In [ ]:
df1.to_csv('prediction.csv',index=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_dtc = DecisionTreeClassifier(random_state=64, max_depth=15)
model_dtc.fit(features_train, target_train)
predictions_dtc = model_dtc.predict(features_test)
model_dtc.score(features_test, target_test)

0.7

In [ ]:
f1_score(predictions_dtc, target_test, average='macro')

0.6827262044653348

In [ ]:
model_dtc.predict(features, check_input=True)

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])